In [1]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import math

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy import spatial

from sentence_transformers import SentenceTransformer
from sentence_transformers import  util

import scipy.spatial

import pandas as pd
import numpy as np


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/abdulwahabmac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Users/abdulwahabmac/Desktop/MyFiles/Projects/Training/Tuwaiq/Tuwaiq-LLM-28-July/.env/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:

model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [9]:

def preprocess_query(query):
    # Tokenization, stop word removal, and stemming
    stop_words = set(stopwords.words('arabic'))
    stemmer = PorterStemmer()
    tokens = query.lower().split()
    filtered_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)


In [10]:

def hybrid_search(query, top_k=3, weight_st=0.5, weight_tfidf=0.5):
    # Encode the query using SentenceTransformers
    query_preprocessed = preprocess_query(query)
    query_embedding = model.encode([query_preprocessed], convert_to_tensor=True)

    # Compute cosine similarities between the query and the documents using SentenceTransformers
    cosine_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]

    # Transform the query using the TF-IDF vectorizer
    query_tfidf = tfidf_vectorizer.transform([query_preprocessed])

    # Calculate cosine similarity between the query and the documents using TF-IDF
    tfidf_similarities = cosine_similarity(query_tfidf, tfidf_matrix)[0]
    #print(tfidf_similarities)

    # Combine the scores with weights
    combined_scores = (weight_st * cosine_scores.cpu().numpy() + weight_tfidf * tfidf_similarities) / (weight_st + weight_tfidf)
    #print(combined_scores)

    # Normalize scores (optional)
    combined_scores = (combined_scores - np.min(combined_scores)) / (np.max(combined_scores) - np.min(combined_scores))

    # Find the top_k highest scores
    top_results = np.argpartition(-combined_scores, range(top_k))[:top_k]
    #print(top_results)

    print(f"Query: {query}\nTop {top_k} most similar arabic_documents:")
    for idx in top_results:
        print(f"Document: {arabic_documents[idx]}, Combined Score: {combined_scores[idx]:.4f}")

    return top_results, combined_scores



In [11]:

# Example documents
arabic_documents  = [
    "حقق الهلال بطولة دوري أبطال آسيا في عام 2019.",
    "تمكن النصر من الفوز بكأس السوبر السعودي في عام 2020.",
    "سجل سالم الدوسري هدفًا رائعًا في نهائي دوري أبطال آسيا لصالح فريق الهلال.",
    "قاد عبدالرزاق حمدالله فريق النصر للفوز بالدوري السعودي في موسم 2018-2019.",
    "تألق محمد كنو في وسط ميدان الهلال وساهم في تحقيق العديد من البطولات.",
    "فاز الاتحاد ببطولة دوري أبطال آسيا مرتين في عامي 2004 و2005.",
    "سجل عمر السومة العديد من الأهداف الحاسمة لصالح فريق الأهلي في الدوري السعودي.",
    "توج الشباب بلقب الدوري السعودي ست مرات في تاريخه.",
    "ساهم ياسر القحطاني في تحقيق الهلال للعديد من البطولات المحلية والقارية.",
    "فاز نادي الاتفاق بكأس الملك في عدة مناسبات."
]

In [12]:

# Encode the documents using SentenceTransformers
document_embeddings = model.encode(arabic_documents, convert_to_tensor=True)

# Create a TF-IDF Vectorizer and fit on the documents
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(arabic_documents)


In [ ]:

# Example usage with evaluation:
while True:
    user_query = input("Enter your query (or type 'exit' to quit): ")
    if user_query.lower() == 'exit':
        break
    hybrid_search(user_query)

    predicted_ranks, _ = hybrid_search(user_query)


Enter your query (or type 'exit' to quit):  افضل نادي


Query: افضل نادي
Top 3 most similar arabic_documents:
Document: فاز نادي الاتفاق بكأس الملك في عدة مناسبات., Combined Score: 1.0000
Document: سجل سالم الدوسري هدفًا رائعًا في نهائي دوري أبطال آسيا لصالح فريق الهلال., Combined Score: 0.1813
Document: سجل عمر السومة العديد من الأهداف الحاسمة لصالح فريق الأهلي في الدوري السعودي., Combined Score: 0.1533
Query: افضل نادي
Top 3 most similar arabic_documents:
Document: فاز نادي الاتفاق بكأس الملك في عدة مناسبات., Combined Score: 1.0000
Document: سجل سالم الدوسري هدفًا رائعًا في نهائي دوري أبطال آسيا لصالح فريق الهلال., Combined Score: 0.1813
Document: سجل عمر السومة العديد من الأهداف الحاسمة لصالح فريق الأهلي في الدوري السعودي., Combined Score: 0.1533
